<a href="https://colab.research.google.com/github/SharooqAtique/Python/blob/main/COVID_19_Dashboard_with_Streamlit_Guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Install required libraries
!pip install plotly pandas requests ipywidgets

# Import libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import requests
from datetime import datetime
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# Set up data caching
from functools import lru_cache

@lru_cache(maxsize=None)
def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

# Data fetching functions
def get_global_data():
    data = fetch_data("https://disease.sh/v3/covid-19/all")
    return {
        'cases': data['cases'],
        'deaths': data['deaths'],
        'recovered': data['recovered'],
        'active': data['active'],
        'updated': datetime.fromtimestamp(data['updated']/1000).strftime('%Y-%m-%d %H:%M')
    } if data else None

def get_country_data():
    data = fetch_data("https://disease.sh/v3/covid-19/countries")
    if data:
        df = pd.DataFrame(data)
        df = df[['country', 'cases', 'todayCases', 'deaths', 'todayDeaths', 'recovered', 'active', 'countryInfo']]
        df['lat'] = df['countryInfo'].apply(lambda x: x['lat'] if 'lat' in x else None)
        df['long'] = df['countryInfo'].apply(lambda x: x['long'] if 'long' in x else None)
        df['flag'] = df['countryInfo'].apply(lambda x: x['flag'] if 'flag' in x else '')
        return df.drop(columns='countryInfo').sort_values('cases', ascending=False)
    return None

def get_historical_data(country="all"):
    url = f"https://disease.sh/v3/covid-19/historical/{country}?lastdays=120"
    data = fetch_data(url)
    if not data:
        return None

    if country == "all":
        cases = data['cases']
    else:
        cases = data['timeline']['cases']

    df = pd.DataFrame(list(cases.items()), columns=['date', 'cases'])
    df['date'] = pd.to_datetime(df['date'])
    df['daily_cases'] = df['cases'].diff().fillna(0)
    df['7day_avg'] = df['daily_cases'].rolling(window=7).mean()
    return df

# Create widgets
country_list = ['Global'] + sorted(get_country_data()['country'].tolist())
country_dropdown = widgets.Dropdown(
    options=country_list,
    value='Global',
    description='Select Country:',
    layout=widgets.Layout(width='300px')
)

date_range = widgets.SelectionSlider(
    options=[30, 60, 90, 120],
    value=90,
    description='Days to Show:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

chart_type = widgets.RadioButtons(
    options=['Daily Cases', 'Total Cases'],
    value='Daily Cases',
    description='Chart Type:',
    disabled=False
)

update_button = widgets.Button(
    description='Update Dashboard',
    button_style='success',
    tooltip='Click to refresh data'
)

# Dashboard layout
header = widgets.HTML("<h1 style='text-align:center; color:#1E3F66;'>🌍 COVID-19 Global Dashboard</h1>")
subheader = widgets.HTML("<p style='text-align:center;'>Real-time tracking powered by disease.sh API</p>")

controls = widgets.HBox(
    [country_dropdown, date_range, chart_type, update_button],
    layout=widgets.Layout(justify_content='space-around', flex_flow='row wrap')
)

metrics_box = widgets.Output()
map_box = widgets.Output()
chart_box = widgets.Output()
table_box = widgets.Output()

dashboard = widgets.VBox([
    header,
    subheader,
    controls,
    metrics_box,
    map_box,
    widgets.HBox([chart_box, table_box])
])

# Display the dashboard framework
display(dashboard)

# Update function
def update_dashboard(change=None):
    # Clear previous outputs
    metrics_box.clear_output()
    map_box.clear_output()
    chart_box.clear_output()
    table_box.clear_output()

    # Get selected values
    country = country_dropdown.value
    days = date_range.value
    show_daily = chart_type.value == 'Daily Cases'

    # Fetch data
    with metrics_box:
        print("⏳ Loading data...")

    global_data = get_global_data()
    country_data = get_country_data()

    # Update metrics
    with metrics_box:
        metrics_box.clear_output()
        if global_data:
            html = f"""
            <div style="display:flex; justify-content:space-around; margin:20px 0; flex-wrap:wrap;">
                <div style="text-align:center; background:#e6f7ff; padding:15px; border-radius:10px; width:23%; min-width:150px; margin:5px;">
                    <h3>🌎 Total Cases</h3>
                    <h2 style="color:#1890ff;">{global_data['cases']:,}</h2>
                </div>
                <div style="text-align:center; background:#fff2e8; padding:15px; border-radius:10px; width:23%; min-width:150px; margin:5px;">
                    <h3>😷 Active Cases</h3>
                    <h2 style="color:#fa8c16;">{global_data['active']:,}</h2>
                </div>
                <div style="text-align:center; background:#fff0f6; padding:15px; border-radius:10px; width:23%; min-width:150px; margin:5px;">
                    <h3>💀 Deaths</h3>
                    <h2 style="color:#eb2f96;">{global_data['deaths']:,}</h2>
                </div>
                <div style="text-align:center; background:#f6ffed; padding:15px; border-radius:10px; width:23%; min-width:150px; margin:5px;">
                    <h3>✅ Recovered</h3>
                    <h2 style="color:#52c41a;">{global_data['recovered']:,}</h2>
                </div>
            </div>
            <p style="text-align:center; color:#666;">Last updated: {global_data['updated']} UTC</p>
            """
            display(HTML(html))

    # Update map
    with map_box:
        if country_data is not None:
            fig = px.scatter_geo(country_data,
                                lat='lat',
                                lon='long',
                                size=np.log10(country_data['cases'] + 1)*10,
                                color='deaths',
                                hover_name='country',
                                hover_data=['cases', 'deaths', 'recovered'],
                                projection="natural earth",
                                title='Global COVID-19 Distribution',
                                color_continuous_scale='Reds')

            if country != 'Global':
                sel_country = country_data[country_data['country'] == country].iloc[0]
                fig.add_trace(go.Scattergeo(
                    lon = [sel_country['long']],
                    lat = [sel_country['lat']],
                    text = [country],
                    mode = 'markers',
                    marker = dict(size=20, color="blue", symbol="star"),
                    name = f"Selected: {country}"
                ))

            fig.update_layout(height=500, title_x=0.5)
            fig.show()

    # Update chart
    with chart_box:
        hist_data = get_historical_data(country if country != 'Global' else "all")
        if hist_data is not None:
            hist_data = hist_data.tail(days)

            title = f"{country} - {'Daily' if show_daily else 'Total'} Cases"
            y_col = 'daily_cases' if show_daily else 'cases'
            y_title = 'Daily New Cases' if show_daily else 'Total Cases'

            fig = px.area(hist_data,
                         x='date',
                         y=y_col,
                         title=title,
                         labels={'date': 'Date', y_col: y_title})

            if show_daily:
                fig.add_scatter(x=hist_data['date'],
                               y=hist_data['7day_avg'],
                               mode='lines',
                               name='7-Day Avg',
                               line=dict(color='red', width=2))

            fig.update_layout(hovermode="x unified", title_x=0.5, height=400)
            fig.show()

    # Update table
    with table_box:
        if country == 'Global':
            display(HTML("<h3 style='text-align:center;'>Top 10 Countries</h3>"))
            display(country_data.head(10).style.format({
                'cases': '{:,}',
                'todayCases': '{:,}',
                'deaths': '{:,}',
                'todayDeaths': '{:,}',
                'recovered': '{:,}',
                'active': '{:,}'
            }).set_properties(**{'text-align': 'left'}))
        else:
            country_stats = country_data[country_data['country'] == country]
            if not country_stats.empty:
                country_stats = country_stats.drop(columns=['lat', 'long'])
                display(HTML(f"<h3 style='text-align:center;'>{country} Statistics</h3>"))
                display(country_stats.style.format({
                    'cases': '{:,}',
                    'todayCases': '{:,}',
                    'deaths': '{:,}',
                    'todayDeaths': '{:,}',
                    'recovered': '{:,}',
                    'active': '{:,}'
                }).set_properties(**{'text-align': 'left'}))
            else:
                print(f"No data available for {country}")

# Set up event handlers
country_dropdown.observe(update_dashboard, names='value')
date_range.observe(update_dashboard, names='value')
chart_type.observe(update_dashboard, names='value')
update_button.on_click(update_dashboard)

# Initial update
update_dashboard()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.4 MB/s eta 0:00:00
